In [1]:
import inspect
import ast
import textwrap

class A:
    def work():
        print("Inside")
        x = 1
        x = x + 1
        x = x + 3
        return x

# Use this to dedent all strings so the first has no indentation, not used
# In this file, just here for if I need it in the future
def remove_common_indentation(strings):
    common_indent = min(len(line) - len(line.lstrip()) for line in strings)
    result = [textwrap.dedent(line)[common_indent:] for line in strings]
    return result
    
# Remove class indentation cause ast.parse() doesnt like it
source_code = textwrap.dedent(inspect.getsource(A.work))
ast_tree = ast.parse(source_code)

# indent=4 to get a nice print
ast_dump = ast.dump(ast_tree, indent=4)
print(f"{ast_dump}")

def is_valid_python_statement(statement):
    try:
        compile(statement, '<string>', 'exec')
        return True
    except SyntaxError:
        return False

source = inspect.getsource(A.work)

function_lines = source.split('\n')
function_lines.pop(0)
function_lines = [line.lstrip() for line in function_lines]
function_lines = [line for line in function_lines if line]

function_lines.insert(2, r'print(f"The value of x is {x}")')
function_lines.insert(4, r'print(f"The value of x is {x}")')
function_lines.insert(6, r'print(f"The value of x is {x}")')

valid_statements = [statement for statement in function_lines if is_valid_python_statement(statement)]

# Need to add the tabs after checking whether the statement is valid, or else
# is_valid_python_statement will return False
valid_statements = [f'\t{string}' for string in valid_statements]

# Execute python code that defines a function and places it in local_vars, then return it.
# Without local_vars I don't have a reference to the defined function afaik
def create_callable_function(valid_statements):
    function_body = '\n'.join(valid_statements)
    local_vars = {}
    exec(f'def function():\n{function_body}', local_vars)
    return local_vars['function']

temp = A.work
A.work = create_callable_function(valid_statements)
t = A.work()
print("Resetting")
A.work = temp
t = A.work()

Module(
    body=[
        FunctionDef(
            name='work',
            args=arguments(
                posonlyargs=[],
                args=[],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Expr(
                    value=Call(
                        func=Name(id='print', ctx=Load()),
                        args=[
                            Constant(value='Inside')],
                        keywords=[])),
                Assign(
                    targets=[
                        Name(id='x', ctx=Store())],
                    value=Constant(value=1)),
                Assign(
                    targets=[
                        Name(id='x', ctx=Store())],
                    value=BinOp(
                        left=Name(id='x', ctx=Load()),
                        op=Add(),
                        right=Constant(value=1))),
                Assign(
                    targets=[
               